# 📊 Interprétabilité et Visualisation des Réseaux de Neurones

## 🎯 Objectifs

Dans ce notebook, nous allons **ouvrir la boîte noire** des réseaux de neurones et comprendre :

- 🧠 **Ce que les neurones apprennent** - Visualisation des poids
- 🔍 **Comment le réseau prend ses décisions** - Activation maps
- 🎨 **Quelles parties des images sont importantes** - Saliency maps
- 📈 **Analyse des erreurs** - Où et pourquoi le modèle se trompe
- 🔬 **Statistiques des activations** - Distribution des neurones

---

## 🤔 Pourquoi l'Interprétabilité est Importante ?

### Les Réseaux de Neurones sont souvent vus comme des **boîtes noires**:

```
Input (image) → [??? MAGIE ???] → Output (prédiction)
```

### Problèmes de cette approche :

1. ❌ **Pas de confiance** : Comment faire confiance à un modèle qu'on ne comprend pas ?
2. ❌ **Difficile à déboguer** : Pourquoi le modèle fait-il des erreurs ?
3. ❌ **Biais cachés** : Le modèle peut apprendre des patterns incorrects
4. ❌ **Pas d'insights** : On ne comprend pas ce que les données nous apprennent

### Solutions : Techniques d'Interprétabilité ✅

1. **Visualisation des poids** → Voir ce que chaque neurone cherche
2. **Activation maps** → Observer les neurones en action
3. **Saliency maps** → Identifier les pixels importants
4. **Analyse des erreurs** → Comprendre les faiblesses
5. **t-SNE / PCA** → Visualiser l'espace des features

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Ajouter le dossier parent au path
sys.path.append('../')

from src.network import NeuralNetwork
from src.utils import load_mnist_data
from src import visualize
from src.metrics import confusion_matrix

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Imports réussis !")

## 1️⃣ Chargement du Modèle Entraîné

Commençons par charger un modèle déjà entraîné.

In [ ]:
# Charger les données
print("Chargement des données MNIST...")
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist_data()

print(f"Train: {X_train.shape[0]:,} exemples")
print(f"Val: {X_val.shape[0]:,} exemples")
print(f"Test: {X_test.shape[0]:,} exemples")

# Charger le modèle entraîné (ou en entraîner un nouveau)
model_path = Path('../models/best_model.pkl')

if model_path.exists():
    print(f"\n📂 Chargement du modèle: {model_path}")
    model = NeuralNetwork.load(model_path)
else:
    print("\n⚠️ Aucun modèle trouvé. Entraînement d'un nouveau modèle...")
    model = NeuralNetwork(
        layer_dims=[784, 256, 128, 10],
        learning_rate=0.01,
        optimizer='adam'
    )
    model.train(X_train, y_train, X_val, y_val, epochs=10, batch_size=128)
    model.save(model_path)

# Évaluer
test_acc = model.accuracy(X_test, y_test)
print(f"\n🎯 Accuracy sur le test: {test_acc:.4f}")

---

## 2️⃣ Visualisation des Poids de la Première Couche

### 🧠 Que font les neurones de la première couche ?

Chaque neurone de la première couche est connecté à **tous les pixels** de l'image (784 connexions).

En visualisant ces poids, on peut voir **ce que chaque neurone cherche** dans l'image.

### Interprétation :

- **Pixels blancs (positifs)** : Le neurone s'active quand ces pixels sont présents
- **Pixels noirs (négatifs)** : Le neurone s'active quand ces pixels sont absents
- **Pixels gris** : Pas d'importance particulière

### Patterns attendus :

Les neurones apprennent souvent à détecter :
- Bords horizontaux/verticaux
- Coins
- Courbes
- Formes géométriques basiques

In [ ]:
# Extraire les poids de la première couche
W1 = model.parameters['W1']  # Shape: (784, 256)

print(f"Poids W1 shape: {W1.shape}")
print(f"Chaque neurone a {W1.shape[0]} poids (un par pixel)")
print(f"Il y a {W1.shape[1]} neurones dans la première couche\n")

# Visualiser 64 neurones
visualize.plot_weights_visualization(
    W1, 
    n_neurons=64,
    figsize=(14, 14),
    save_path='../models/weights_layer1.png'
)

### 🔍 Analyse des Poids

**Question** : Que voyez-vous ?

Observez attentivement :

1. **Certains neurones détectent des bords** (horizontaux, verticaux, diagonaux)
2. **D'autres détectent des formes** (courbes, coins)
3. **Certains sont spécialisés** pour des zones précises de l'image
4. **Quelques-uns semblent bruyants** (pas de pattern clair) - c'est normal !

C'est fascinant : **le réseau découvre automatiquement** ces features à partir des données ! 🤯

In [ ]:
# Analyser les statistiques des poids
print("📊 Statistiques des Poids W1:\n")
print(f"Mean: {W1.mean():.6f}")
print(f"Std:  {W1.std():.6f}")
print(f"Min:  {W1.min():.6f}")
print(f"Max:  {W1.max():.6f}")

# Distribution des poids
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Histogramme
ax1.hist(W1.flatten(), bins=100, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='Zéro')
ax1.set_xlabel('Valeur du poids', fontsize=12)
ax1.set_ylabel('Fréquence', fontsize=12)
ax1.set_title('Distribution des Poids (Couche 1)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Heatmap de corrélation entre neurones
sample_neurons = W1[:, :50].T  # Prendre 50 neurones
correlation = np.corrcoef(sample_neurons)
sns.heatmap(correlation, cmap='coolwarm', center=0, square=True, ax=ax2,
            cbar_kws={'label': 'Corrélation'})
ax2.set_title('Corrélation entre Neurones (50 premiers)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Interprétation:")
print("- Distribution centrée sur 0 → Bonne initialisation He")
print("- Faible corrélation entre neurones → Chaque neurone apprend des features différentes")

---

## 3️⃣ Visualisation des Activations (Neurones en Action)

### 🔥 Activation Maps

Maintenant, voyons comment les neurones **réagissent** à de vraies images.

Pour une image donnée :
- Certains neurones s'activent fortement (valeurs élevées)
- D'autres restent silencieux (valeurs proches de 0)

Cela nous montre **quels neurones sont activés** pour chaque type de chiffre.

In [ ]:
# Fonction pour extraire les activations intermédiaires
def get_layer_activations(model, X):
    """
    Récupère les activations de toutes les couches
    """
    _, cache = model.forward(X)
    
    activations = {}
    for l in range(1, len(model.layer_dims)):
        activations[f'Layer {l}'] = cache[f'A{l}']
    
    return activations

# Prendre quelques exemples
n_samples = 5
sample_indices = np.random.choice(len(X_test), n_samples, replace=False)
X_samples = X_test[sample_indices]
y_samples = y_test[sample_indices]

# Obtenir les activations
activations = get_layer_activations(model, X_samples)

print("Activations récupérées:")
for layer_name, acts in activations.items():
    print(f"  {layer_name}: shape {acts.shape}")

In [ ]:
# Visualiser les activations pour chaque couche
layer_names = [f'Layer {i}' for i in range(1, len(model.layer_dims)-1)]  # Exclure la sortie

visualize.plot_activation_outputs(
    activations,
    layer_names,
    n_samples=3,
    n_neurons=32,
    figsize=(15, 8),
    save_path='../models/activations.png'
)

In [ ]:
# Analyser la sparsité des activations (après ReLU)
print("\n📊 Analyse de la Sparsité (% de neurones actifs):\n")

for layer_name, acts in activations.items():
    if 'Layer' in layer_name and layer_name != f'Layer {len(model.layer_dims)-1}':
        # Compter les neurones actifs (> 0)
        active_neurons = (acts > 0).sum() / acts.size * 100
        print(f"{layer_name}: {active_neurons:.1f}% neurones actifs")

print("\n💡 Une sparsité de ~50% est idéale (grâce à ReLU)")
print("   - Trop peu (<20%) : Les neurones ne s'activent pas assez")
print("   - Trop (>80%) : Risque de surapprentissage")

---

## 4️⃣ Saliency Maps : Pixels Importants

### ❓ Question Clé :

**Quels pixels de l'image ont le plus d'influence sur la décision du réseau ?**

### 🎯 Saliency Map (Carte de Saillance)

Une **saliency map** montre l'importance de chaque pixel pour la prédiction.

**Méthode** : Calculer le gradient de la sortie par rapport à l'input

```
Saliency = |∂output / ∂input|
```

- **Valeurs élevées** → Le pixel est important
- **Valeurs faibles** → Le pixel n'a pas d'impact

### Utilité :

✅ Vérifier que le réseau se concentre sur les **bonnes parties** de l'image  
✅ Détecter si le réseau apprend des **biais** (ex: se concentrer sur le fond plutôt que le chiffre)  
✅ Comprendre les **erreurs** du modèle

In [ ]:
def compute_saliency_map(model, x, target_class):
    """
    Calcule la saliency map pour une image
    
    Args:
        model: réseau de neurones
        x: image (784,)
        target_class: classe cible
    
    Returns:
        saliency: gradient de l'output par rapport à l'input (784,)
    """
    # Forward pass
    x_batch = x.reshape(1, -1)
    output, cache = model.forward(x_batch)
    
    # Créer un gradient pour la classe cible
    grad_output = np.zeros_like(output)
    grad_output[0, target_class] = 1.0
    
    # Backprop simple (sans changer les poids)
    # Gradient de la couche de sortie
    dZ = grad_output
    
    # Backprop à travers les couches
    L = len(model.layer_dims) - 1
    
    for l in reversed(range(1, L + 1)):
        dA_prev = np.dot(dZ, model.parameters[f'W{l}'].T)
        
        # Si pas la première couche, appliquer dérivée ReLU
        if l > 1:
            dZ = dA_prev * (cache[f'Z{l-1}'] > 0)
        else:
            # Gradient par rapport à l'input
            saliency = dA_prev
    
    # Valeur absolue (on s'intéresse à la magnitude)
    saliency = np.abs(saliency).reshape(784)
    
    return saliency

print("✓ Fonction saliency_map définie")

In [ ]:
# Calculer et visualiser les saliency maps
n_examples = 10
sample_indices = np.random.choice(len(X_test), n_examples, replace=False)

fig, axes = plt.subplots(3, n_examples, figsize=(16, 6))

for i, idx in enumerate(sample_indices):
    x = X_test[idx]
    y_true = y_test[idx]
    
    # Prédiction
    pred = model.predict(x.reshape(1, -1))[0]
    
    # Saliency map
    saliency = compute_saliency_map(model, x, pred)
    
    # Image originale
    axes[0, i].imshow(x.reshape(28, 28), cmap='gray')
    axes[0, i].set_title(f'Vrai: {y_true}\nPred: {pred}', fontsize=9)
    axes[0, i].axis('off')
    
    # Saliency map
    axes[1, i].imshow(saliency.reshape(28, 28), cmap='hot')
    axes[1, i].set_title('Saliency', fontsize=9)
    axes[1, i].axis('off')
    
    # Superposition
    axes[2, i].imshow(x.reshape(28, 28), cmap='gray', alpha=0.7)
    axes[2, i].imshow(saliency.reshape(28, 28), cmap='hot', alpha=0.3)
    axes[2, i].set_title('Overlay', fontsize=9)
    axes[2, i].axis('off')

axes[0, 0].set_ylabel('Image', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Saliency Map', fontsize=12, fontweight='bold')
axes[2, 0].set_ylabel('Superposition', fontsize=12, fontweight='bold')

plt.suptitle('🔍 Saliency Maps - Pixels Importants pour la Décision', 
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig('../models/saliency_maps.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Interprétation:")
print("Les zones rouges/jaunes sont les pixels importants pour la décision.")
print("Le réseau se concentre-t-il sur le chiffre ou sur le fond ?")

---

## 5️⃣ Analyse des Erreurs

### 🐛 Pourquoi le Modèle se Trompe-t-il ?

Analyser les erreurs est **crucial** pour améliorer le modèle.

**Questions importantes** :

1. Quels chiffres sont les plus confondus ?
2. Les erreurs sont-elles compréhensibles (même pour un humain) ?
3. Y a-t-il des patterns dans les erreurs ?
4. Le modèle est-il confiant quand il se trompe ?

In [ ]:
# Faire des prédictions sur le test set
print("Calcul des prédictions sur le test set...")
y_pred = model.predict(X_test)
y_probs, _ = model.forward(X_test)

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred, num_classes=10)

print("\n📊 Matrice de Confusion:\n")
visualize.plot_confusion_matrix(
    cm,
    class_names=[str(i) for i in range(10)],
    figsize=(12, 5),
    save_path='../models/confusion_matrix_analysis.png'
)

In [ ]:
# Identifier les paires les plus confondues
print("\n🔍 Top 10 Confusions:\n")

confusions = []
for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 0:
            confusions.append((i, j, cm[i, j]))

# Trier par nombre de confusions
confusions.sort(key=lambda x: x[2], reverse=True)

print(f"{'Vrai':<6} {'Prédit':<8} {'Erreurs':<10} {'% du vrai'}")
print("-" * 40)

for true, pred, count in confusions[:10]:
    total_true = cm[true, :].sum()
    percentage = (count / total_true) * 100
    print(f"{true:<6} {pred:<8} {count:<10} {percentage:.2f}%")

print("\n💡 Ces confusions sont-elles logiques ?")
print("   Ex: 3/5, 4/9, 7/1 sont visuellement similaires")

In [ ]:
# Visualiser les exemples mal classifiés (avec haute confiance)
print("\n🚨 Exemples Mal Classifiés (avec haute confiance):\n")

visualize.plot_misclassified_examples(
    X_test,
    y_test,
    y_pred,
    y_probs,
    n_samples=20,
    figsize=(14, 12),
    save_path='../models/misclassified_confident.png'
)

In [ ]:
# Analyser la distribution de confiance
errors_indices = np.where(y_test != y_pred)[0]
correct_indices = np.where(y_test == y_pred)[0]

# Confiance sur les prédictions
confidence_errors = np.max(y_probs[errors_indices], axis=1)
confidence_correct = np.max(y_probs[correct_indices], axis=1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogrammes de confiance
ax1.hist(confidence_correct, bins=50, alpha=0.7, color='green', 
         label=f'Correctes ({len(correct_indices)})', edgecolor='black')
ax1.hist(confidence_errors, bins=50, alpha=0.7, color='red', 
         label=f'Erreurs ({len(errors_indices)})', edgecolor='black')
ax1.set_xlabel('Confiance (max probabilité)', fontsize=12)
ax1.set_ylabel('Fréquence', fontsize=12)
ax1.set_title('Distribution de Confiance', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Box plot
data_to_plot = [confidence_correct, confidence_errors]
ax2.boxplot(data_to_plot, labels=['Correctes', 'Erreurs'], 
            patch_artist=True,
            boxprops=dict(facecolor='lightblue', color='black'),
            medianprops=dict(color='red', linewidth=2))
ax2.set_ylabel('Confiance', fontsize=12)
ax2.set_title('Comparaison de Confiance', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../models/confidence_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Statistiques de Confiance:\n")
print(f"Prédictions Correctes:")
print(f"  Mean: {confidence_correct.mean():.4f}")
print(f"  Median: {np.median(confidence_correct):.4f}")
print(f"\nPrédictions Incorrectes:")
print(f"  Mean: {confidence_errors.mean():.4f}")
print(f"  Median: {np.median(confidence_errors):.4f}")

# Erreurs avec haute confiance (> 0.9)
high_conf_errors = np.sum(confidence_errors > 0.9)
print(f"\n⚠️ Erreurs avec confiance > 90%: {high_conf_errors} ({high_conf_errors/len(errors_indices)*100:.1f}%)")
print("   → Le modèle est très sûr de lui même quand il se trompe !")

---

## 6️⃣ Visualisation de l'Espace des Features (t-SNE)

### 🗺️ t-SNE : Réduction de Dimensionnalité

**t-SNE** (t-Distributed Stochastic Neighbor Embedding) projette les données haute dimension en 2D.

### Pourquoi c'est utile ?

- Les features apprises par le réseau sont en haute dimension (ex: 128D)
- Impossible de les visualiser directement
- t-SNE les projette en 2D tout en **préservant les similarités**

### Interprétation :

✅ **Clusters bien séparés** → Le réseau a bien appris à séparer les classes  
❌ **Clusters mélangés** → Le réseau confond certaines classes  
🔍 **Points isolés** → Exemples difficiles ou outliers

In [ ]:
# Installer sklearn si nécessaire
try:
    from sklearn.manifold import TSNE
except ImportError:
    print("Installation de scikit-learn...")
    !pip install scikit-learn
    from sklearn.manifold import TSNE

print("✓ t-SNE disponible")

In [ ]:
# Extraire les features de l'avant-dernière couche
# (c'est là que les représentations sont les plus riches)
n_samples_tsne = 1000  # Limiter pour la vitesse
sample_indices = np.random.choice(len(X_test), n_samples_tsne, replace=False)
X_sample = X_test[sample_indices]
y_sample = y_test[sample_indices]

# Forward pass pour obtenir les features
_, cache = model.forward(X_sample)
features = cache[f'A{len(model.layer_dims)-2}']  # Avant-dernière couche

print(f"Features extraites: {features.shape}")
print(f"Dimension originale: {features.shape[1]}D")
print(f"\nCalcul du t-SNE (peut prendre 1-2 minutes)...")

# Appliquer t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
features_2d = tsne.fit_transform(features)

print("✓ t-SNE terminé !")

In [ ]:
# Visualiser le t-SNE
fig, ax = plt.subplots(figsize=(12, 10))

# Couleurs pour chaque classe
colors = plt.cm.tab10(np.linspace(0, 1, 10))

for digit in range(10):
    mask = y_sample == digit
    ax.scatter(features_2d[mask, 0], features_2d[mask, 1],
              c=[colors[digit]], label=str(digit),
              alpha=0.6, edgecolors='black', linewidth=0.5, s=50)

ax.set_xlabel('t-SNE Dimension 1', fontsize=12, fontweight='bold')
ax.set_ylabel('t-SNE Dimension 2', fontsize=12, fontweight='bold')
ax.set_title('Visualisation t-SNE des Features Apprises', fontsize=14, fontweight='bold')
ax.legend(title='Chiffre', loc='best', frameon=True, shadow=True)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../models/tsne_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Interprétation:")
print("- Clusters bien séparés → Le réseau distingue bien les classes")
print("- Chevauchements → Chiffres visuellement similaires (ex: 3/5, 4/9)")
print("- Points isolés → Exemples atypiques ou mal écrits")

---

## 🎓 Résumé et Conclusions

### 🔑 Points Clés de l'Interprétabilité

1. **Visualisation des Poids** 🧠
   - Montre ce que chaque neurone cherche dans l'image
   - Révèle des détecteurs de bords, coins, courbes
   - Permet de vérifier que le réseau apprend des features sensées

2. **Activation Maps** 🔥
   - Montre quels neurones s'activent pour chaque image
   - Analyse de la sparsité (idéalement ~50% avec ReLU)
   - Permet de détecter les neurones "morts" ou trop actifs

3. **Saliency Maps** 🎯
   - Identifie les pixels importants pour la décision
   - Vérifie que le réseau se concentre sur les bonnes zones
   - Détecte les biais potentiels

4. **Analyse des Erreurs** 🐛
   - Comprendre où et pourquoi le modèle se trompe
   - Identifier les confusions logiques (3/5, 4/9, etc.)
   - Analyser la confiance (le modèle sait-il quand il ne sait pas ?)

5. **Visualisation t-SNE** 🗺️
   - Projette les features en 2D
   - Montre la séparation des classes dans l'espace appris
   - Révèle les exemples difficiles et outliers

---

### 🚀 Applications Pratiques

Ces techniques d'interprétabilité sont **essentielles** pour :

✅ **Debugging** : Trouver pourquoi le modèle ne performe pas  
✅ **Confiance** : Comprendre et faire confiance aux prédictions  
✅ **Amélioration** : Identifier les faiblesses et les corriger  
✅ **Transparence** : Expliquer les décisions (crucial en médecine, finance, etc.)  
✅ **Détection de biais** : S'assurer que le modèle est équitable

---

### 💡 Pour Aller Plus Loin

**Techniques avancées d'interprétabilité** :

1. **Grad-CAM** : Version améliorée des saliency maps pour les CNN
2. **LIME** : Explication locale avec modèles linéaires
3. **SHAP** : Valeurs de Shapley pour l'importance des features
4. **Adversarial Examples** : Trouver les faiblesses du modèle
5. **Feature Visualization** : Générer des images qui maximisent l'activation

**Bibliothèques utiles** :
- `captum` (PyTorch)
- `tf-explain` (TensorFlow)
- `lime`
- `shap`

---

## 🎯 Exercices

1. **Modifier l'architecture** et observer l'impact sur les poids visualisés
2. **Comparer les saliency maps** entre prédictions correctes et incorrectes
3. **Identifier les neurones morts** (jamais activés) dans votre réseau
4. **Analyser les erreurs** : Y a-t-il des patterns spécifiques ?
5. **t-SNE sur différentes couches** : Comment l'espace des features évolue ?

---

**Félicitations ! 🎉**

Vous savez maintenant **ouvrir la boîte noire** et comprendre ce qui se passe à l'intérieur de vos réseaux de neurones !